A comparison between LR and NWAY methods
======================================

We crossmatch XXL-North X-ray sources with SDSS optical sources, both selected within a 1 deg radius circle, using the LR and NWAY methods, and compare the results.

Definition of the catalogues and set crossmatch:

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from mocpy import MOC

from astromatch import Catalogue, Match


# Common MOC
region_center = SkyCoord(ra=35.30109, dec=-4.45962, unit='deg')
search_radius = 1*u.deg

moc_xxl = MOC.from_elliptical_cone(
    lon=region_center.ra,
    lat=region_center.dec,
    a=search_radius,
    b=search_radius,
    pa=0*u.deg,
    max_depth=14
)

# X-rays
xcat = Catalogue(
    'xxl_1deg.fits',
    name='xxl',
    id_col='Xseq',
    coord_cols=['RABdeg', 'DEBdeg'],
    poserr_cols=['e_Bpos'],
    poserr_type='circle',
    area=moc_xxl,
)

# Optical
ocat = Catalogue(
    'sdss_1deg.fits',
    name='sdss',
    id_col='objID',
    coord_cols=['RA_ICRS', 'DE_ICRS'],
    poserr_cols=['e_RA_ICRS', 'e_DE_ICRS'],
    poserr_type='rcd_dec_ellipse',
    area=moc_xxl,
    mag_cols=['umag', 'gmag', 'rmag', 'imag', 'zmag'],
)

xm = Match(xcat, ocat)

Cross-matching using LR method:

In [ ]:
results_lr = xm.run(method='lr', radius=10.0*u.arcsec)
matchs_lr_primary = xm.get_matchs(match_type='primary')

Save calculated magnitude priors for later use:

In [ ]:
prior_table = xm.priors.to_table(include_bkg_priors=True)
prior_table.write("lrpriors.fits", format="fits", overwrite=True)

Cross-matching using NWAY (default magnitude priors):

In [ ]:
results_nway_dfprior = xm.run(method='nway', radius=10.0*u.arcsec, use_mags=True, prior_completeness=0.55)
matchs_nway_dfprior_primary = xm.get_matchs(match_type='primary')

Cross-matching using NWAY (LR magnitude priors):

In [ ]:
from astromatch.priors import Prior

priors = {'sdss': Prior.from_table("lrpriors.fits", ['umag', 'gmag', 'rmag', 'imag', 'zmag'])}

results_nway_lrprior = xm.run(
    method='nway', radius=10.0*u.arcsec, prior_completeness=0.55, use_mags=True, priors=priors
)
matchs_nway_lrprior_primary = xm.get_matchs(match_type='primary')

Comparison of results
-------------------------------

Identification rate:

In [ ]:
lr_id_rate = 100 * len(matchs_lr_primary) / len(xcat)
print(f"LR identification rate :{lr_id_rate:.1f}%")

lr_nway_dfprior_rate = 100 * len(matchs_nway_dfprior_primary) / len(xcat)
print(f"NWAY (default prior) identification rate :{lr_nway_dfprior_rate:.1f}%")

lr_nway_lrprior_rate = 100 * len(matchs_nway_lrprior_primary) / len(xcat)
print(f"NWAY (LR prior) identification rate :{lr_nway_lrprior_rate:.1f}%")


prob_has_match distributions:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(9,7))

_, bins, _ = plt.hist(matchs_lr_primary["prob_has_match"], label="LR",
         bins="auto")
plt.hist(matchs_nway_dfprior_primary["prob_has_match"], label="NWAY (default prior)",
         histtype='step', lw=5, ls=":", bins=bins)
plt.hist(matchs_nway_lrprior_primary["prob_has_match"], label="NWAY (LR prior)",
         histtype='step', lw=5, ls="--", bins=bins)

plt.legend()

plt.xlim(0, 1)
plt.xlabel("prob_has_match")

plt.show()

In [ ]:
import numpy as np
from astropy.table import join

matchs_lr_primary_ids = matchs_lr_primary[["SRCID_xxl", "SRCID_sdss"]]
matchs_nway_dfprior_primary_ids = matchs_nway_dfprior_primary[["SRCID_xxl", "SRCID_sdss"]]
matchs_nway_lrprior_primary_ids = matchs_nway_lrprior_primary[["SRCID_xxl", "SRCID_sdss"]]

lr_nway_dfprior = join(matchs_lr_primary_ids, matchs_nway_dfprior_primary_ids,
                       join_type="left", keys=["SRCID_xxl"], table_names=["lr", "nway"])

equal_ids = len(np.where(lr_nway_dfprior["SRCID_sdss_lr"] == lr_nway_dfprior["SRCID_sdss_nway"])[0])
percent_equal_ids = 100 * equal_ids / len(lr_nway_dfprior)

print(f"LR and NWAY (default prior) get the same counterpart for {percent_equal_ids:.1f}% of the X-ray sources.")


lr_nway_lrprior = join(matchs_lr_primary_ids, matchs_nway_lrprior_primary_ids,
                       join_type="left", keys=["SRCID_xxl"], table_names=["lr", "nway"])

equal_ids = len(np.where(lr_nway_lrprior["SRCID_sdss_lr"] == lr_nway_lrprior["SRCID_sdss_nway"])[0])
percent_equal_ids = 100 * equal_ids / len(lr_nway_lrprior)

print(f"LR and NWAY (LR prior) get the same counterpart for {percent_equal_ids:.1f}% of the X-ray sources.")